In [34]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
# import pandas as pd
from nltk import pos_tag
from nltk.stem import PorterStemmer

# libraries for dataset preparation, feature engineering
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd
from sklearn import model_selection, preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups


%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

np.random.seed(123) #for reprodicible results
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WCL-2017\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\WCL-2017\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\WCL-2017\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\WCL-2017\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [35]:
newsgroups_train = fetch_20newsgroups(subset='train')
x_train = newsgroups_train.data

In [36]:
newsgroups_test = fetch_20newsgroups(subset='test')
x_test = newsgroups_test.data

In [37]:
y_train = newsgroups_train.target
y_test = newsgroups_test.target

In [38]:
def preprocessing(text):
    text2 = " ".join("".join([" " if ch in string.punctuation else ch for ch in text]).split())
    tokens = [word for sent in nltk.sent_tokenize(text2) for word in nltk.word_tokenize(sent)]
    tokens = [word.lower() for word in tokens]
    stopwds = stopwords.words('english')
    tokens = [token for token in tokens if token not in stopwds]
    tokens = [word for word in tokens if len(word)>=3]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    tagged_corpus = pos_tag(tokens)
    Noun_tags = ['NN','NNP','NNPS','NNS']
    Verb_tags = ['VB','VBD','VBG','VBN','VBP','VBZ']
    lemmatizer = WordNetLemmatizer()
    def prat_lemmatize(token,tag):
        if tag in Noun_tags:
            return lemmatizer.lemmatize(token,'n')
        elif tag in Verb_tags:
            return lemmatizer.lemmatize(token,'v')
        else:
            return lemmatizer.lemmatize(token,'n')
    pre_proc_text = " ".join([prat_lemmatize(token,tag) for token,tag in tagged_corpus])
    return pre_proc_text

In [40]:
x_train_preprocessed = []
for i in x_train:
    x_train_preprocessed.append(preprocessing(i))
x_test_preprocessed = []
for i in x_test:
    x_test_preprocessed.append(preprocessing(i))

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1, 2), stop_words='english', max_features= 10000,strip_accents='unicode', norm='l2')
x_train_2 = vectorizer.fit_transform(x_train_preprocessed).todense()
x_test_2 = vectorizer.transform(x_test_preprocessed).todense()

In [42]:
# Deep Learning modules
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adadelta,Adam,RMSprop
from keras.utils import np_utils

In [43]:
# Definition hyper parameters
np.random.seed(1337)
nb_classes = 20
batch_size = 64
nb_epochs = 20
Y_train = np_utils.to_categorical(y_train, nb_classes)

In [44]:
# Deep Layer Model building in Keras
model = Sequential()
model.add(Dense(1000,input_shape= (10000,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# model.add(Dense(500))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 1000)              10001000  
_________________________________________________________________
activation_8 (Activation)    (None, 1000)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 50)                50050     
_________________________________________________________________
activation_9 (Activation)    (None, 50)                0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 20)                1020      
__________

In [45]:
# Model Training
model.fit(x_train_2, Y_train, batch_size=batch_size, epochs=nb_epochs,verbose=1)

Epoch 1/20
11314/11314 [==============================] - 37s 3ms/step - loss: 1.8887
Epoch 2/20
11314/11314 [==============================] - 35s 3ms/step - loss: 0.6054
Epoch 3/20
11314/11314 [==============================] - 35s 3ms/step - loss: 0.3266
Epoch 4/20
11314/11314 [==============================] - 36s 3ms/step - loss: 0.2143
Epoch 5/20
11314/11314 [==============================] - 38s 3ms/step - loss: 0.1550
Epoch 6/20
11314/11314 [==============================] - 37s 3ms/step - loss: 0.1148
Epoch 7/20
11314/11314 [==============================] - 37s 3ms/step - loss: 0.0929
Epoch 8/20
11314/11314 [==============================] - 37s 3ms/step - loss: 0.0763
Epoch 9/20
11314/11314 [==============================] - 37s 3ms/step - loss: 0.0656
Epoch 10/20
11314/11314 [==============================] - 36s 3ms/step - loss: 0.0554
Epoch 11/20
11314/11314 [==============================] - 36s 3ms/step - loss: 0.0461
Epoch 12/20
11314/11314 [===========================

In [46]:
#Model Prediction

y_train_predclass = model.predict_classes(x_train_2,batch_size=batch_size)
y_test_predclass = model.predict_classes(x_test_2,batch_size=batch_size)
from sklearn.metrics import accuracy_score,classification_report
print ("nnDeep Neural Network - Train accuracy: %f" % (round(accuracy_score( y_train, y_train_predclass),3))),
print ("nDeep Neural Network - Test accuracy: %f" % (round(accuracy_score( y_test,y_test_predclass),3))),
print ("nDeep Neural Network - Train Classification Report")
print (classification_report(y_train,y_train_predclass))
print ("nDeep Neural Network - Test Classification Report")
print (classification_report(y_test,y_test_predclass))

nnDeep Neural Network - Train accuracy: 0.999000
nDeep Neural Network - Test accuracy: 0.820000
nDeep Neural Network - Train Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       480
           1       1.00      1.00      1.00       584
           2       1.00      1.00      1.00       591
           3       1.00      1.00      1.00       590
           4       1.00      1.00      1.00       578
           5       1.00      1.00      1.00       593
           6       1.00      1.00      1.00       585
           7       1.00      1.00      1.00       594
           8       1.00      1.00      1.00       598
           9       1.00      1.00      1.00       597
          10       1.00      1.00      1.00       600
          11       1.00      1.00      1.00       595
          12       1.00      1.00      1.00       591
          13       1.00      1.00      1.00       594
          14       1.00      1.00      1.0